In [2]:
import requests
from bs4 import BeautifulSoup
from persiantools.jdatetime import JalaliDate


In [ ]:
cookies = {
    'Province': '8',
    'City': '95',
    'ASP.NET_SessionId': 'teufzlt0gijv0wawu1jyqy4k',
    '__AntiXsrfToken': '1977b87c4a7244198ad779bf7a74406a',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:126.0) Gecko/20100101 Firefox/126.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    # 'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Referer': 'https://www.time.ir/fa/eventyear-%d8%aa%d9%82%d9%88%db%8c%d9%85-%d8%b3%d8%a7%d9%84%db%8c%d8%a7%d9%86%d9%87',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'https://www.time.ir',
    'DNT': '1',
    'Sec-GPC': '1',
    'Connection': 'keep-alive',
    # 'Cookie': 'Province=8; City=95; ASP.NET_SessionId=teufzlt0gijv0wawu1jyqy4k; __AntiXsrfToken=1977b87c4a7244198ad779bf7a74406a',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Priority': 'u=1',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

data = {
    'ctl00_rssmStyleSheet_TSSM': '',
    'ctl00_rsmAll_TSM': '',
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': 'KvrgsEKsqdBEYviEKrXdg9HuFa/iOvQq9dnVYPOJQMJYGSqy+xcJFXRiedfO51obpKlJ507u5CGzJ5KBwqbo8sQj16/HiahBlvR0Hzc6RUZciVdy',
    'ctl00$cphTop$Sampa_Web_View_EventUI_EventYearCalendar10cphTop_3417$txtYear': '1402',
    'ctl00$cphTop$Sampa_Web_View_EventUI_EventYearCalendar10cphTop_3417$btnGo': 'برو به سال',
    '__VIEWSTATEGENERATOR': '65E7F3AF',
}

response = requests.post(
    'https://www.time.ir/fa/eventyear-%D8%AA%D9%82%D9%88%DB%8C%D9%85-%D8%B3%D8%A7%D9%84%DB%8C%D8%A7%D9%86%D9%87',
    cookies=cookies,
    headers=headers,
    data=data,
)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
soup


<!DOCTYPE html>

<html itemscope="" itemtype="http://schema.org/WebPage" lang="fa" xml:lang="fa">
<head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="LF3TA" name="fontiran.com:license"/><meta content="UECNB" name="fontiran.com:license"/><meta content="PN88C" name="fontiran.com:license"/><title>
	تقویم سالیانه | تایم
</title>
<meta content="index, follow" name="robots"/>
<link href="/App_Themes/favicon.ico" rel="shortcut icon" type="image/x-icon"/><meta content="ساعت و تقویم ایران" name="apple-mobile-web-app-title"/><link href="/App_Themes/touch-icon-iphone.png" rel="apple-touch-icon"/><link href="/App_Themes/touch-icon-ipad.png" rel="apple-touch-icon" sizes="76x76"/><link href="/App_Themes/touch-icon-iphone-retina.png" rel="apple-touch-icon" sizes="120x120"/><link href="/App_Themes/touch-icon-ipad-retina.png" rel="apple-touch-icon" sizes="152x152"/>
<meta content="https:/

In [ ]:
a = soup.find_all('div', class_='holiday')[1]
a

<div class="holiday"><div class="jalali" style="">۱</div><div class="miladi small" style="float:right">21</div><div class="qamari small" style="float:left">٢٨</div><div class="clearBoth"></div></div>

In [ ]:
d = a.find('div', class_='jalali').text
d

'۱'

In [ ]:
'disabled' not in a.parent['class']

True

In [ ]:
tmp = a.parent.parent.parent.parent.find('a', class_='jalali').text
tmp

'فروردین ۱۴۰۲'

In [ ]:
m, y = tmp.split(' ')
y, m

('۱۴۰۲', 'فروردین')

In [ ]:
int(y)

1402

In [ ]:
month = {
    'فروردین': 1,
    'اردیبهشت': 2,
    'خرداد': 3,
    'تیر': 4,
    'مرداد': 5,
    'شهریور': 6,
    'مهر': 7,
    'آبان': 8,
    'آذر': 9,
    'دی': 10,
    'بهمن': 11,
    'اسفند': 12
}
month[m]

1

In [ ]:
int(d)

1

In [ ]:
JalaliDate(int(y), month[m], int(d))

JalaliDate(1402, 1, 1, Seshanbeh)